In [72]:
import requests
import json # Make sure json is imported

OLLAMA_API_CHAT="http://localhost:11434/api/chat" # or OLLAMA_API_GENERATE
MODEL="llama3.2:1b"
HEADERS={"Content-type" : "application/json"} # This header will be set automatically by `json=`

prompt="Tell me somehing about TVK political party in Tamil Nadu, India"
request_body = {
    "model" : MODEL,
    "prompt" : prompt,
    # Add 'stream': False if you want to ensure a single response and not a stream
    "stream": False
}

try:
    r = requests.post(OLLAMA_API_CHAT, json=request_body) # <--- CHANGE IS HERE: use json=
    r.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

    # Process the response based on the endpoint
    if OLLAMA_API_CHAT.endswith("/api/chat"):
        full_response = ""
        # The /api/chat endpoint streams responses, even if stream=False is not explicit
        # It sends one JSON object per line.
        # For non-streaming (stream=False), it's still line-delimited, but usually one line.
        # You might need to read the lines if it sends multiple.
        for line in r.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                try:
                    json_data = json.loads(decoded_line)
                    if "content" in json_data["message"]:
                         full_response += json_data["message"]["content"]
                    # For the final response in a streamed chat, you might also get 'done' field
                    if json_data.get("done"):
                        break # Stop processing if the response is complete
                except json.JSONDecodeError:
                    print(f"Warning: Could not decode JSON line: {decoded_line}")
                    continue
        print("Model's Full Response:")
        print(full_response)

    elif OLLAMA_API_CHAT.endswith("/api/generate"): # If you're using the generate endpoint
        response_data = r.json()
        print("Model's Response:")
        print(response_data.get("response", "No response content found."))

except requests.exceptions.ConnectionError as e:
    print(f"Connection Error: Could not connect to Ollama at {OLLAMA_API_CHAT}. "
          f"Please ensure Ollama is running (`ollama serve` or `ollama run <model>`) "
          f"and accessible.")
    print(e)
except requests.exceptions.HTTPError as e:
    print(f"HTTP Error: {e}")
    print(f"Status Code: {r.status_code}")
    print(f"Response Text: {r.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

HTTP Error: 404 Client Error: Not Found for url: http://localhost:11434/api/chat
Status Code: 404
Response Text: {"error":"model \"llama3.2:1b\" not found, try pulling it first"}


In [1]:
import ollama
response = ollama.generate(model = 'llama3.2', prompt = "Why the sky is blue?")
print(response)

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download